# COVID-19 Analysis Platform

In [950]:
from jupyter_dash import JupyterDash

In [951]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import dash_bootstrap_components as dbc
import dash_table as dt
from dash.dependencies import Input, Output

In [952]:
debug=True

In [953]:
df = pd.read_csv("owid-covid-data.csv")

In [954]:
countries = df.location.unique()

In [955]:
dpm_countries = 20

dft = df[df.groupby(['location']).date.transform('max') == df.date]
dft = dft[['location', 'continent', 'total_cases', 'new_cases', 'total_deaths', 'new_deaths']]


In [956]:
header = html.H1("CovAP dashboard")
last_updated = html.H6("Last updated on: "+df.date.max(),className="text-right")

In [957]:
# Daily stats tab content
daily_stats_div = html.Div([
    
    html.Div(html.Div(html.Label("Select country"), className="col"), className="row"),
        
    html.Div(html.Div(dcc.Dropdown(
            id='country-input',
            options=[{'label': i, 'value': i} for i in countries],
            value='Norway',
            clearable=False,
    ), className="col-3"), className="row"),
    
    html.Div([
        
        html.Div(dcc.Graph(id='daily-cases'), className="col"),
        
        html.Div(dcc.Graph(id='daily-deaths'), className="col"),
        
    ], className="row"),
    
], className="container-fluid")

In [958]:
# World data tab cotent
world_data_div = html.Div([
    
    html.Div(html.Div(html.H2("Latest infection data"), className="col"), className="row"),
    
    html.Div(html.Div(
        dt.DataTable(
        data=dft.to_dict('records'),
        columns=[{'id': c, 'name': c.title().replace('_', ' ')} for c in dft.columns],
        page_size=10,
        sort_action='native',
        filter_action="native",
        sort_by=[{
            "column_id": 'total_cases',
            "direction": "desc"
                }],
        style_cell_conditional=[
        {
            'if': {'column_id': c},
            'textAlign': 'left'
        } for c in ['Date', 'Region']
        ],
        style_data_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(248, 248, 248)'
            },
            {
            'if': {
                'filter_query': '{location} = "World"'
            },
            'backgroundColor': 'grey',
            'color': 'white'
            }
        ],
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
        }), className="col"), className="row"),
    
], className="container-fluid")

In [959]:
# Analysis tab content
analysis_div = html.Div([
    
    html.Div(html.Div(
        html.H2("Analysis of top " +str(dpm_countries)+" countries with highest deaths per million"),
        className="col"), className="row"),
    
    html.Div(html.Div(dcc.RadioItems(
                id='parameter',
                options=[{'label': i, 'value': i} for i in ['Hospital beds per thousand', 'Aged 65 or older(%)']],
                value='Hospital beds per thousand',
#                 labelStyle={'display': 'inline-block'}
            ), className="col"), className="row"),
    
    html.Div(html.Div(dcc.Graph(id='deaths-per-million'), className="col"), className="row"),
    
], className="container-fluid")

In [960]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Create server variable with Flask server object for use with gunicorn
server = app.server

app.layout = html.Div([
    html.Div([html.Div(header, className="col"), html.Div(last_updated,className="col")], className="row"),
    html.Div(html.Div(dcc.Tabs([
        dcc.Tab(label='Daily Statistics', children=[
            daily_stats_div
        ]),
        dcc.Tab(label='World Data', children=[
            world_data_div
        ]),
        dcc.Tab(label='Analysis', children=[
            analysis_div
        ]),
        dcc.Tab(label='Dataset', children=[
        ]),
    ]),className="col"),className="row"),
], className="container-fluid")

In [961]:
# Daily stats tab callback
@app.callback([
    Output('daily-cases', 'figure'),
    Output('daily-deaths', 'figure'),
],
[
    Input('country-input', 'value'),
])
def update_daily_stats(country):
    
    dff = df[df.location == country]
   
    figure1 =  {
        'data': [dict(
            x=dff.date,
            y=dff.new_cases,
            type='bar',
            marker={
                'color': 'grey',
            }
        )],
        'layout': dict(
            margin={'l': 40, 'b': 30, 't': 60, 'r': 0},
            hovermode='closest',
            title={
            'text': "Daily new cases",
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': dict(size=20)
            }
        )
    }
    
    figure2 =  {
        'data': [dict(
            x=dff.date,
            y=dff.new_deaths,
            type='bar',
            marker={
                'color': 'grey'
            }
        )],
        'layout': dict(
            margin={'l': 40, 'b': 30, 't': 60, 'r': 0},
            hovermode='closest',
            title={
            'text': "Daily new deaths",
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': dict(size=20)
            }
        )
    }    
    return figure1, figure2


In [962]:
# Analysis tab callback
@app.callback(
    dash.dependencies.Output('deaths-per-million', 'figure'),
[
    dash.dependencies.Input('parameter', 'value')
])
def update_analysis(parameter):
    test = df[['location', 'date', 'total_deaths_per_million', 'hospital_beds_per_thousand', 'aged_65_older', 'population']]
    test = test[test.groupby(['location']).date.transform('max') == df.date]
    data = test.dropna().reset_index().sort_values("total_deaths_per_million", ascending=False).head(dpm_countries)
    
    y = data.hospital_beds_per_thousand
    if(parameter == "Aged 65 or older(%)"):
        y = data.aged_65_older
    
    figure =  {
        'data': [
                dict(
            x=data.location,
            y=y,
#             name=parameter,
#             customdata='Total deaths per million: '+str(data.total_deaths_per_million),
            type='bar',
#             mode='markers',
            marker={
#                 'size': 25,
#                 'opacity': 0.7,
                'color': 'orange',
#                 'line': {'width': 2, 'color': 'purple'}
            },
            hovertext="Total deaths per million: "+data.total_deaths_per_million.round(2).astype(str),
        )],
        'layout': dict(
#             margin={'l': 40, 'b': 30, 't': 10, 'r': 0},
#             height=450,
#             hovermode='closest',
            title= parameter
        )
    }
    
    return figure


In [963]:
app.run_server(debug=debug)

Dash app running on http://127.0.0.1:8050/
